<a href="https://colab.research.google.com/github/405620294/iShape/blob/main/training_waist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# connect your google drive，please follow the steps
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# before run this ipynb, u need to notice that:
# 1. dataframe are sort as follows ：['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']
# 2. choose a Normalization which u want to do at KNN Regression block，　and all the linear egression are use the same scaler
# 3. before predict data，please define model_name = the algorithm which u want to use

# github dataset >> train 8 model to predict waist by 8 algorithm


## raw ( original dataset ) >> df ( preprocessed dataset ) >> df_input ( filtered column dataframe ) >> slicing target ( x_df / y_df )

In [36]:
import pandas as pd
bw_df =  pd.read_csv("/content/drive/Shareddrives/AI_Python/ishape/ai/step1/hps_output/body_width.csv", encoding="utf-8")

# data preprocessing

In [37]:
### preprocessing  ###

bw_df = bw_df[['org_img', 'age', 'gender', 'race', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'chest', 'waist', 'hips', 'bfp', 'source']]

# missing value >> delete all
bw_df.dropna(axis=0, how='any', inplace=True)

# gender m:0 f:1 >> don't need to one-hot-encoding
dic = { 'm' : 0, 'f' : 1}
bw_df['gender'] = bw_df['gender'].replace(dic)

# race e:0 w:1 >> don't need to one-hot-encoding
dic = { 'e' : 0, 'w' : 1}
bw_df['race'] = bw_df['race'].replace(dic)

# height(cm) >> m^2
def t_m2(t):
  t = t / 100
  return t * t
m2 = bw_df['height'].apply(t_m2)

# add column bmi
bw_df['bmi'] = bw_df['weight'] / m2

print('bw_df has', len(bw_df), 'rows')

bw_df has 262 rows


# data screening

In [38]:
### condition screening (Select the data for training) ###
df = bw_df

# filter gender column(m:0 f:1)
# delete_idx = df.loc[df["gender"] == 0].index.tolist() # 去掉以下欄位
# df = df.drop(index=delete_idx)

# filter race column(e:0 w:1)
# delete_idx = df.loc[df["race"] == 1].index.tolist() # 去掉以下欄位
# df = df.drop(index=delete_idx)

# filter source column
c = '水下秤重'
delete_idx = df.loc[df["source"] != c].index.tolist() 
df = df.drop(index=delete_idx)

print('df has', len(df), 'rows')

df has 251 rows


# column screening

In [39]:
### column delete (delete the column don't need to training) ###

df_input = df.drop(['bfp', 'org_img', 'gender', 'race', 'source'], axis=1)

print('complete the selection of data， df_input has', len(df_input), 'row')
print("df_input' s columns :", df_input.columns)

complete the selection of data， df_input has 251 row
df_input' s columns : Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'chest', 'waist', 'hips', 'bmi'],
      dtype='object')


# preparation of training data & data slice

In [40]:
x_df = df_input[['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']] # 欄位排序
y_df = df_input['waist']

In [41]:
print(' training data: x_df has', len(x_df), 'rows, and columns : ', x_df.columns)
print(' training data: y_df has', len(y_df), 'rows, and columns : ', y_df.name)

 training data: x_df has 251 rows, and columns :  Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object')
 training data: y_df has 251 rows, and columns :  waist


In [42]:
# data slice
import numpy as np
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.array(x_df), np.array(y_df), test_size=0.1)

In [43]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score

# Regression Tree

In [44]:
### dt ###

from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()
params = {"max_depth":range(3, 10)} # the search range of Grid Search
grid = GridSearchCV(dt, params, cv=10, n_jobs=-1, scoring="neg_root_mean_squared_error")
grid.fit(x_df, y_df)
print('the best max depth of DecisionTreeRegressor is ', grid.best_params_)

dt = DecisionTreeRegressor(max_depth = grid.best_params_['max_depth'])
dt.fit(x_df, y_df)
pre = dt.predict(x_df)
print('dt_r2: ', r2_score(y_df, pre))
print('dt_MAE: ', mean_absolute_error(y_df, pre))
print('dt_MSE: ', mean_squared_error(y_df, pre))

dt = DecisionTreeRegressor(max_depth = grid.best_params_['max_depth'])
# Evaluate a score by cross-validation
dt_cross_r2 = cross_val_score(dt, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
dt_cross_RMSE = cross_val_score(dt, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')


dt_dic = {
    'name' : 'dt',
    'R2' : np.average(dt_cross_r2),
    'RMSE' : np.average(dt_cross_RMSE)
}

dt.fit(x_df, y_df)

the best max depth of DecisionTreeRegressor is  {'max_depth': 6}
dt_r2:  0.9709593351812661
dt_MAE:  1.2408517386386986
dt_MSE:  3.0183233846841477


DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=6,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

# Random Forests Regression

In [45]:
### rf ###

from sklearn.ensemble import RandomForestRegressor

# find the best estimators & max depth of rf
rf = RandomForestRegressor()
params = {
    "n_estimators":range(10, 101, 2),
    "max_depth":range(3, 10)
}
grid = GridSearchCV(rf, params, cv=10, n_jobs=-1, scoring="neg_root_mean_squared_error")
grid.fit(x_df, y_df)
print('best_score : ', grid.best_score_, 'the best params of DecisionTreeRegressor : ', grid.best_params_)


rf = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'], max_depth = grid.best_params_['max_depth'])

# Evaluate a score by cross-validation
rf_cross_r2 = cross_val_score(rf, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
rf_cross_RMSE = cross_val_score(rf, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

rf_dic = {
    'name' : 'rf',
    'R2' : np.average(rf_cross_r2),
    'RMSE' : np.average(rf_cross_RMSE)
}

rf.fit(x_df, y_df)

best_score :  -3.089085344228818 the best params of DecisionTreeRegressor :  {'max_depth': 8, 'n_estimators': 40}


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=8, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=40, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [46]:
#找出 隨機森林特徵的重要性
pd.DataFrame(rf.feature_importances_, 
       columns=["Importance"],
       index=x_df.keys()).sort_values(by="Importance", ascending=False)
       #ascending=False降冪排列

,Importance
waist_width_cm,0.685939
bmi,0.225813
weight,0.041125
chest_width_cm,0.020556
age,0.010157
hip_width_cm,0.009651
height,0.006760


# SVR Regression

In [47]:
### svr regression ###
from sklearn.svm import SVR
linear_svr = SVR(kernel='linear') 
poly_svr = SVR(kernel='poly') 
rbf_svr = SVR(kernel='rbf', C=100)

# Evaluate a score by cross-validation
linear_svr_r2 = cross_val_score(linear_svr, np.array(x_df), np.array(y_df), cv=10, scoring="r2") 
linear_svr_RMSE = cross_val_score(linear_svr, np.array(x_df), np.array(y_df), cv=10, scoring="neg_root_mean_squared_error")

linear_svr_dic = {
    'name' : 'linear_svr',
    'R2' : np.average(linear_svr_r2),
    'RMSE' : np.average(linear_svr_RMSE)
}

poly_svr_r2 = cross_val_score(poly_svr, np.array(x_df), np.array(y_df), cv=10, scoring="r2") 
poly_svr_RMSE = cross_val_score(poly_svr, np.array(x_df), np.array(y_df), cv=10, scoring="neg_root_mean_squared_error") 

poly_svr_dic = {
    'name' : 'poly_svr',
    'R2' : np.average(poly_svr_r2),
    'RMSE' : np.average(poly_svr_RMSE)
}

rbf_svr_r2 = cross_val_score(rbf_svr, np.array(x_df), np.array(y_df), cv=10, scoring="r2") 
rbf_svr_RMSE = cross_val_score(rbf_svr, np.array(x_df), np.array(y_df), cv=10, scoring="neg_root_mean_squared_error")

rbf_svr_dic = {
    'name' : 'rbf_svr',
    'R2' : np.average(rbf_svr_r2),
    'RMSE' : np.average(rbf_svr_RMSE)
}

linear_svr.fit(x_df, y_df)
poly_svr.fit(x_df, y_df)
rbf_svr.fit(x_df, y_df)

SVR(C=100, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

# KNN Regression

In [48]:
# prepare another dataset for KNN & linear ( cause data needs to Normalization before training) 
from sklearn.preprocessing import RobustScaler, MaxAbsScaler, MinMaxScaler, StandardScaler
def do_scaler(name, input_df):
  scaler = name()
  scaler.fit(input_df)
  return scaler

# choose a Normalization which u want to do by yourself !
scaler = MinMaxScaler # u can choose : RobustScaler, MaxAbsScaler, MinMaxScaler, StandardScaler

df_need_scaler = df_input.drop(['chest', 'waist', 'hips'], axis=1)
knn_scaler = do_scaler(scaler, df_need_scaler) # fit the scaler
scalered_df = pd.DataFrame(knn_scaler.transform(df_need_scaler), columns = df_need_scaler.columns) # transform the dataframe
print("the'", df_need_scaler.columns , "'column，already do", scaler)

the' Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object') 'column，already do <class 'sklearn.preprocessing._data.MinMaxScaler'>


In [49]:
x_df = scalered_df[['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']] # colimn sort
y_df = df_input['waist']

print(' training data: x_df has', len(x_df), 'rows, and columns : ', x_df.columns)
print(' training data: y_df has', len(y_df), 'rows, and columns : ', y_df.name)

 training data: x_df has 251 rows, and columns :  Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object')
 training data: y_df has 251 rows, and columns :  waist


In [50]:
x_train, x_test, y_train, y_test = train_test_split(np.array(x_df), np.array(y_df), test_size=0.1)

In [51]:
### knn ###
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
params = {"n_neighbors":range(1, 50)}
grid = GridSearchCV(knn, params, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')
grid.fit(x_df, y_df)
print('the best n of KNN is ', grid.best_params_)

# Evaluate a score by cross-validation
knn_cross_r2 = cross_val_score(knn, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
knn_cross_RMSE = cross_val_score(knn, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

knn_dic = {
    'name' : 'knn',
    'R2' : np.average(knn_cross_r2),
    'RMSE' : np.average(knn_cross_RMSE)
}

knn.fit(x_df, y_df)

the best n of KNN is  {'n_neighbors': 7}


KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

# Linear Regression

In [52]:
# prepare another dataset cause datas' distribution needs to be normal before trainning any Linear Regression)

# defined function for find the skewness of each columns
from scipy.stats import skew
def getskew(series):
  return skew(series.dropna() + 1)

# defined function for adjust skewed to normal
from scipy.stats import boxcox_normmax
def get_lamda(s):
  lamda = boxcox_normmax(s.dropna() + 1)
  return lamda

from scipy.special import boxcox1p, inv_boxcox1p
def sk_normal(s):
  lamda = boxcox_normmax(s.dropna() + 1)
  return boxcox1p(s, lamda)


# check the the skewness of each columns
column = df_input.columns 
skewness = (df_input[column].apply(getskew).sort_values(ascending=False))
print(skewness)

# get each columns' lamda & let series change to dictionary
lamdas = df_input[column].apply(get_lamda)
lamdas = lamdas.to_dict()

# adjust skewed to normal
df_input[column] = df_input[column].apply(sk_normal)
skewness = (df_input[column].apply(getskew).sort_values(ascending=False))
print('after adjust skewed')
print(skewness)

bmi               0.713082
hip_width_cm      0.543655
hips              0.493493
chest             0.478005
chest_width_cm    0.468446
waist             0.405025
weight            0.358957
waist_width_cm    0.331246
age               0.282339
height            0.110810
dtype: float64
after adjust skewed
age               0.023249
hip_width_cm      0.021719
waist_width_cm    0.008794
chest             0.004731
waist             0.002116
weight            0.000183
height           -0.002526
bmi              -0.005223
hips             -0.009498
chest_width_cm   -0.010964
dtype: float64


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3538: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [53]:
# prepare target variable and Scaler input variable

df_need_scaler = df_input.drop(['chest', 'waist', 'hips'], axis=1)
# you already choosed the Normalization at the KNN Regression block
linear_scaler = do_scaler(scaler, df_need_scaler) # fit the scaler
scalered_df = pd.DataFrame(linear_scaler.fit_transform(df_need_scaler), columns = df_need_scaler.columns) # transform t
print("the'", df_need_scaler.columns , "'column，already do", scaler)

the' Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object') 'column，already do <class 'sklearn.preprocessing._data.MinMaxScaler'>


In [54]:
x_df = scalered_df[['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']] # colimn sort
y_df = df_input['waist']

print(' training data: x_df has', len(x_df), 'rows, and columns : ', x_df.columns)
print(' training data: y_df has', len(y_df), 'rows, and columns : ', y_df.name)

 training data: x_df has 251 rows, and columns :  Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object')
 training data: y_df has 251 rows, and columns :  waist


In [55]:
x_train, x_test, y_train, y_test = train_test_split(np.array(x_df), np.array(y_df), test_size=0.1)

In [56]:
# linear ###
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear_cross_r2 = cross_val_score(linear, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
linear_cross_RMSE = cross_val_score(linear, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

linear_dic = {
    'name' : 'linear',
    'R2' : np.average(linear_cross_r2),
    'RMSE' : np.average(linear_cross_RMSE)
}

linear.fit(x_df, y_df)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [57]:
coeffecients = pd.DataFrame(linear.coef_,x_df.columns)
coeffecients.columns = ['Coeffecient'] 
coeffecients

,Coeffecient
age,0.005017
height,-0.019704
weight,0.050307
chest_width_cm,-0.049109
waist_width_cm,0.094294
hip_width_cm,0.001582
bmi,-0.022743


# Lasso Regression

In [58]:
### lasso ### 
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV
from sklearn.linear_model import Lasso, Ridge, ElasticNet

cv = LassoCV(cv=10)
cv.fit(x_df, y_df)
print('LassoCV', cv.alpha_)
lasso = Lasso(cv.alpha_)
lasso_cross_r2 = cross_val_score(lasso, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
lasso_cross_RMSE = cross_val_score(lasso, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

lasso_dic = {
    'name' : 'lasso',
    'R2' : np.average(lasso_cross_r2),
    'RMSE' : np.average(lasso_cross_RMSE)
}

lasso.fit(x_df, y_df)

LassoCV 4.284269821794573e-06


Lasso(alpha=4.284269821794573e-06, copy_X=True, fit_intercept=True,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

# Ridge Regression

In [59]:
### ridge ###

cv = RidgeCV(cv=10)
cv.fit(x_df, y_df)
print(cv.alpha_)

ridge = Ridge(cv.alpha_)
ridge_cross_r2 = cross_val_score(ridge, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
ridge_cross_RMSE = cross_val_score(ridge, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

ridge_dic = {
    'name' : 'ridge',
    'R2' : np.average(ridge_cross_r2),
    'RMSE' : np.average(ridge_cross_RMSE)
}

ridge.fit(x_df, y_df)

0.1


Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

# Elastic Net Regression

In [60]:
### ElasticNet ###
cv = ElasticNetCV(cv=10)
cv.fit(x_df, y_df)
print(cv.alpha_)

elastic = ElasticNet(cv.alpha_)

elastic_cross_r2 = cross_val_score(elastic, x_df, y_df, cv=10, n_jobs=-1, scoring='r2')
elastic_cross_RMSE = cross_val_score(elastic, x_df, y_df, cv=10, n_jobs=-1, scoring='neg_root_mean_squared_error')

elastic_dic = {
    'name' : 'elastic',
    'R2' : np.average(elastic_cross_r2),
    'RMSE' : np.average(elastic_cross_RMSE)
}

elastic.fit(x_df, y_df)

7.991048749512792e-06


ElasticNet(alpha=7.991048749512792e-06, copy_X=True, fit_intercept=True,
           l1_ratio=0.5, max_iter=1000, normalize=False, positive=False,
           precompute=False, random_state=None, selection='cyclic', tol=0.0001,
           warm_start=False)

# cross-validation

In [61]:
table = pd.DataFrame.from_dict(dt_dic,orient='index').T
for i in [rf_dic, linear_svr_dic, poly_svr_dic, rbf_svr_dic, knn_dic, linear_dic, lasso_dic, ridge_dic, elastic_dic]:
  a = pd.DataFrame.from_dict(i, orient='index').T
  table = pd.concat([table, a], axis=0, ignore_index=True)

In [62]:
# cross-validation ( CV = 10 ) 

print('linear, Lasso, Ridge and ElasticNet had adjusted skewed to normal')
print('knn, linear, Lasso, Ridge and ElasticNet had adjusted the range with' , scaler)
print(' training data: x_df has', len(x_df), 'rows, and columns : ', x_df.columns)
print(' training data: y_df has', len(y_df), 'rows, and columns : ', y_df.name)
table

linear, Lasso, Ridge and ElasticNet had adjusted skewed to normal
knn, linear, Lasso, Ridge and ElasticNet had adjusted the range with <class 'sklearn.preprocessing._data.MinMaxScaler'>
 training data: x_df has 251 rows, and columns :  Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object')
 training data: y_df has 251 rows, and columns :  waist


,name,R2,RMSE
0,dt,0.773627,-3.74871
1,rf,0.830648,-3.18012
2,linear_svr,0.903871,-2.514
3,poly_svr,0.783393,-3.75317
4,rbf_svr,0.847276,-3.03619
5,knn,0.765462,-3.94665
6,linear,0.907894,-0.00373026
7,lasso,0.907232,-0.00374042
8,ridge,0.899952,-0.0038755
9,elastic,0.907213,-0.00374101


# predict data

In [63]:
### please adjust 'model_name' by yourself ! ###

# u can choose : dt, rf, linear_svr, poly_svr, rbf_svr, knn, linear, lasso, ridge, elastic
model_name = rf 
algorithm = str(model_name).split('(')[0]

In [64]:
### condition screening (Select the data for predict) ###
df = bw_df

# filter gender column(m:0 f:1)
delete_idx = df.loc[df["gender"] == 1].index.tolist() # selete
df = df.drop(index=delete_idx)

print('df has', len(df), 'rows')

df has 261 rows


In [65]:
### column delete (delete the column don't need) ###

predict_df = df[['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm', 'hip_width_cm', 'bmi']]

print('complete the selection of data， df_input has', len(predict_df), 'row')
print("predict_df' s columns :", predict_df.columns)

complete the selection of data， df_input has 261 row
predict_df' s columns : Index(['age', 'height', 'weight', 'chest_width_cm', 'waist_width_cm',
       'hip_width_cm', 'bmi'],
      dtype='object')


In [66]:
###　adjust skewed to normal ###

need_adjust_skewed = ['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']

if algorithm in need_adjust_skewed :
  for i in predict_df.keys():
    predict_df[i]= boxcox1p(predict_df[i], lamdas[i])
  print('the algorithm u use now is', algorithm, ', predict_df complete skewness adjustment')

else:
  print('the algorithm u use now is', algorithm, ", so predict_df don't need to skewness adjustment")

the algorithm u use now is RandomForestRegressor , so predict_df don't need to skewness adjustment


In [67]:
###　scaler ###
need_scaler = ['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']

if algorithm == 'KNeighborsRegressor':
  predict_df = pd.DataFrame(knn_scaler.transform(predict_df), columns = predict_df.columns)
  print('the algorithm u use now is', algorithm, ' predict_df complete', scaler)

elif algorithm in need_scaler:
  predict_df = pd.DataFrame(linear_scaler.transform(predict_df), columns = predict_df.columns)
  print('the algorithm u use now is', algorithm, ' predict_df complete', scaler)

else:
  print('the algorithm u use now is', algorithm, "so predict_df don't need to scaler")

the algorithm u use now is RandomForestRegressor so predict_df don't need to scaler


In [68]:
### predict ###

pre = model_name.predict(predict_df).round(4)

need_inv_skewed = ['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']

if algorithm in need_inv_skewed:
  pre = inv_boxcox1p(pre, lamdas['waist'])
  print('the algorithm u use now is', algorithm, ' predictive value complete skewness reduction',)
else:
  print('the algorithm u use now is', algorithm, "so predictive value don't need to skewness reduction")

the algorithm u use now is RandomForestRegressor so predictive value don't need to skewness reduction


In [69]:
### check results & output dataframe ###

output = df
# adjust column order
number_col = output.pop(df.columns[10])
output.insert(14, number_col.name, number_col)

output['pre_waist'] = pre
output['pre_waist - real_waist'] = (output['pre_waist'] - output['waist']).abs()

# gender m:0 f:1 
dic = { 0 : '男性', 1 : '女性'}
output['gender'] = output['gender'].replace(dic)

# race e:0 w:1 
dic = { 0 : '東方人', 1 : '西方人'}
output['race'] = output['race'].replace(dic)

print('the algorithm u use now is', algorithm, ' , and use:　', scaler)
print('complete predict， output has', len(output), 'rows')

the algorithm u use now is RandomForestRegressor  , and use:　 <class 'sklearn.preprocessing._data.MinMaxScaler'>
complete predict， output has 261 rows


In [70]:
### loss of bfp > 5 ###
print('loss of bfp > 5 has', len(output[output['pre_waist - real_waist'] >= 5]), 'rows')
output[output['pre_waist - real_waist'] >= 5].sort_values(['pre_waist - real_waist'], ascending=False)

loss of bfp > 5 has 4 rows


,org_img,age,gender,race,height,weight,chest_width_cm,waist_width_cm,hip_width_cm,chest,hips,bfp,source,bmi,waist,pre_waist,pre_waist - real_waist
419,8704.jpg,24.0,男性,東方人,161.5,50.5,36.355,33.512,36.056,77.0,88.0,9.2,真人,19.361827,66.5,86.0987,19.5987
406,8691.jpg,26.0,男性,東方人,176.0,59.0,35.278,33.522,36.874,83.0,91.0,14.3,真人,19.047004,74.0,86.2211,12.2211
418,8703.jpg,29.0,男性,東方人,182.0,72.0,35.490,35.976,38.407,88.0,96.5,17.2,真人,21.736505,83.0,91.1159,8.1159
417,8702.jpg,24.0,男性,東方人,168.0,92.7,38.232,34.934,40.332,99.5,110.0,32.0,真人,32.844388,89.0,96.5508,7.5508


In [71]:
### real people ###
output[output['source'] == '真人'].sort_values(['pre_waist - real_waist'], ascending=False)

,org_img,age,gender,race,height,weight,chest_width_cm,waist_width_cm,hip_width_cm,chest,hips,bfp,source,bmi,waist,pre_waist,pre_waist - real_waist
419,8704.jpg,24.0,男性,東方人,161.5,50.5,36.355,33.512,36.056,77.0,88.0,9.2,真人,19.361827,66.5,86.0987,19.5987
406,8691.jpg,26.0,男性,東方人,176.0,59.0,35.278,33.522,36.874,83.0,91.0,14.3,真人,19.047004,74.0,86.2211,12.2211
418,8703.jpg,29.0,男性,東方人,182.0,72.0,35.490,35.976,38.407,88.0,96.5,17.2,真人,21.736505,83.0,91.1159,8.1159
417,8702.jpg,24.0,男性,東方人,168.0,92.7,38.232,34.934,40.332,99.5,110.0,32.0,真人,32.844388,89.0,96.5508,7.5508
405,8690.jpg,24.0,男性,東方人,165.0,48.1,31.133,27.241,35.997,76.5,84.0,11.9,真人,17.667585,68.0,72.9894,4.9894
414,8699.jpg,28.0,男性,東方人,168.0,67.6,37.740,32.049,37.440,90.0,95.0,21.2,真人,23.951247,81.0,85.5254,4.5254
410,8695.jpg,29.0,男性,東方人,172.0,72.9,36.602,34.125,34.125,95.0,99.0,21.9,真人,24.641698,87.0,90.8250,3.8250
407,8692.jpg,23.0,男性,東方人,179.0,98.7,36.892,36.892,42.291,103.0,114.0,26.1,真人,30.804282,105.0,101.8225,3.1775
404,8689.jpg,29.0,男性,東方人,166.0,64.8,34.913,31.118,39.011,87.0,95.0,22.6,真人,23.515750,82.0,84.8966,2.8966
408,8693.jpg,29.0,男性,東方人,174.0,73.0,34.555,31.614,38.819,91.0,99.0,21.1,真人,24.111507,87.0,87.4150,0.4150


# savw model
save the model to Google drive

In [72]:
# import time
# from sklearn.externals import joblib
# import datetime
# import os
# import json

# now = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S') # 當前時間(注意為伺服器時間)
# print('當前時間', now)

# folder = algorithm # 以演算法區分資料夾
# drive_path = os.path.join('/content/drive/Shareddrives/AI_Python/ishape/ai/step2/waist', folder)

# if not os.path.exists(drive_path):
#   os.makedirs(drive_path)

# # save model(.h5)
# name_model = folder + '_pre_waist_' + str(now) + '.h5'
# joblib.dump(model_name, os.path.join(drive_path, name_model))

# if algorithm in ['KNeighborsRegressor', 'LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']:

#   if algorithm == 'KNeighborsRegressor':
#     save_scaler = knn_scaler
#   elif algorithm in ['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']:
#     save_scaler = linear_scaler

#     # 存skewness
#     name_lamda = folder + '_lamda_' + str(now) + '.json'
#     f = open(os.path.join(drive_path, name_lamda), "w", encoding="UTF-8") #寫入
#     json.dump(lamdas , f, ensure_ascii=False, indent=4) #輸出成json檔案
#     f.close()
#     print('已儲存lamdas 參數')


#   # 存scaler
#   name_scaler = folder + '_' + str(save_scaler).split('(')[0] + '_' + str(now) + '.h5'
#   joblib.dump(save_scaler, os.path.join(drive_path, name_scaler))
#   print('已儲存scaler model')

# save results
save the results to Google drive (same as model' s path

In [73]:
# ### save the result into Excel ###
# import os

# table_name = name_model.split('.')[0] + '.xlsx' # Ridge_pre_bodyfat_2021-05-13_10:34:31.xlsx
# writer = pd.ExcelWriter(os.path.join(drive_path, table_name), engine='openpyxl')
# sheets =  y_df.name + '_' + str(len(df_input)) +  '_' + algorithm
# output.to_excel(writer, sheet_name = sheets, index=False, encoding='utf-8')
# writer.save()